In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import random
import os

In [ ]:
#def get_jobs(keyword, num_jobs, verbose=False):
#    """Scrapes Glassdoor job postings and returns them as a Pandas DataFrame."""
#    
#    # Create screenshots directory if it doesn't exist
#    if not os.path.exists("screenshots"):
#        os.makedirs("screenshots")
#    
#    # Setup Chrome options
#    options = webdriver.ChromeOptions()
#    options.add_argument("--start-maximized")  # Open in full-screen mode
#    options.add_argument("--disable-popup-blocking")  # Block popups
#    # Add a user agent to appear more like a regular browser
#    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36")
#
#    # Initialize WebDriver with webdriver-manager
#    service = Service(ChromeDriverManager().install())
#    driver = webdriver.Chrome(service=service, options=options)
#    driver.set_window_size(1366, 768)
#
#    # Try different URL formats
#    urls = [
#        f'https://www.glassdoor.ca/Job/canada-{keyword.replace(" ", "-")}-jobs-SRCH_IL.0,6_IN3_KO7,{7+len(keyword)}.htm',
#        f'https://www.glassdoor.com/Job/jobs.htm?sc.keyword={keyword}&locT=C&locId=1147401&locKeyword=Canada',
#        f'https://www.glassdoor.com/Job/canada-{keyword.replace(" ", "-")}-jobs-SRCH_IL.0,6_IN3_KO7,{7+len(keyword)}.htm'
#    ]
#    
#    # Try each URL until one works
#    for url in urls:
#        try:
#            print(f"Trying URL: {url}")
#            driver.get(url)
#            time.sleep(5)  # Wait for initial page load
#            
#            # Debugging info
#            print(f"Page title: {driver.title}")
#            print("Page source length:", len(driver.page_source))
#            driver.save_screenshot("screenshots/initial_page.png")
#            
#            # Check if we hit a CAPTCHA or login wall
#            if "Verify" in driver.title or "Sign In" in driver.title or "Security Challenge" in driver.page_source:
#                print("Detected verification page or login requirement")
#                driver.save_screenshot("screenshots/verification_page.png")
#                continue  # Try the next URL
#                
#            # Initial page loaded successfully
#            break
#        except Exception as e:
#            print(f"Error with URL {url}: {e}")
#            continue
#    else:
#        print("All URL attempts failed")
#        driver.quit()
#        return pd.DataFrame()
#
#    # Try to close any initial popups that might appear
#    try:
#        popup_selectors = [
#            "//button[contains(@class, 'close')]",
#            "//button[contains(@class, 'CloseButton')]",
#            "//span[contains(@class, 'CloseButton')]",
#            "//div[contains(@class, 'modal_closeIcon')]",
#            "//button[contains(text(), 'Close')]",
#            "//button[contains(text(), 'Continue')]",
#            "//button[contains(text(), 'Accept')]"
#        ]
#        
#        for selector in popup_selectors:
#            try:
#                print(f"Looking for popup with selector: {selector}")
#                close_button = WebDriverWait(driver, 3).until(
#                    EC.element_to_be_clickable((By.XPATH, selector))
#                )
#                close_button.click()
#                print(f"Closed popup with selector: {selector}")
#                time.sleep(1)
#            except (TimeoutException, NoSuchElementException):
#                continue
#    except Exception as e:
#        print(f"Error handling popups: {e}")
#
#    # Look for job listings with different potential selectors
#    print("Looking for job listings...")
#    job_card_class = None
#    potential_selectors = [
#        "react-job-listing",
#        "JobCard",
#        "css-109wfo5",  # Check actual class names from the page
#        "jobCard",
#        "job-listing-card",
#        "jobListItem",
#        "css-7yd1sz",
#        "css-fksa8w"
#    ]
#    
#    for selector in potential_selectors:
#        try:
#            print(f"Trying selector: {selector}")
#            elements = WebDriverWait(driver, 8).until(
#                EC.presence_of_all_elements_located((By.CLASS_NAME, selector))
#            )
#            if len(elements) > 0:
#                print(f"Found {len(elements)} elements with selector: {selector}")
#                job_card_class = selector
#                break
#        except:
#            continue
#    
#    if job_card_class is None:
#        print("Could not find job listings with any known selector")
#        driver.save_screenshot("screenshots/no_job_cards.png")
#        # Try XPath as a last resort
#        try:
#            job_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'job') or contains(@data-test, 'job')]")
#            if len(job_elements) > 0:
#                print(f"Found {len(job_elements)} job elements with generic XPath")
#                # We'll handle these elements differently
#                job_card_class = "xpath_fallback"
#            else:
#                driver.quit()
#                return pd.DataFrame()
#        except:
#            driver.quit()
#            return pd.DataFrame()
#
#    jobs = []
#    page_num = 1
#
#    while len(jobs) < num_jobs:
#        print(f"Scraping page {page_num}... Jobs collected so far: {len(jobs)}")
#        driver.save_screenshot(f"screenshots/page_{page_num}.png")
#
#        # Try to close any popups again
#        try:
#            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CLASS_NAME, "selected"))).click()
#            print("Closed signup popup.")
#        except:
#            pass  # Ignore if no popup found
#
#        # Get job listings
#        try:
#            if job_card_class == "xpath_fallback":
#                job_cards = driver.find_elements(By.XPATH, "//div[contains(@class, 'job') or contains(@data-test, 'job')]")
#            else:
#                job_cards = driver.find_elements(By.CLASS_NAME, job_card_class)
#            
#            print(f"Found {len(job_cards)} job cards on this page.")
#        except NoSuchElementException:
#            print("Could not find job cards.")
#            break
#
#        for i, job_card in enumerate(job_cards):
#            if len(jobs) >= num_jobs:
#                break
#
#            # Add some randomness to appear more human-like
#            time.sleep(random.uniform(0.5, 1.5))
#            
#            try:
#                print(f"Clicking job card {i+1}")
#                driver.execute_script("arguments[0].scrollIntoView(true);", job_card)
#                job_card.click()  # Click on the job card to get details
#                time.sleep(random.uniform(1.0, 2.0))
#            except Exception as e:
#                print(f"Error clicking job card: {e}")
#                continue
#
#            # Try multiple selector patterns for job details
#            try:
#                # Company name selectors
#                company_selectors = [
#                    ".css-87uc0g", 
#                    ".css-1vg0w6e", 
#                    ".css-l2wjgv", 
#                    "[data-test='employer-name']",
#                    "//div[contains(@class, 'employer')]",
#                    "//div[contains(@class, 'company')]"
#                ]
#                
#                # Use a function to try multiple selectors
#                def try_selectors(selectors, element_type):
#                    for selector in selectors:
#                        try:
#                            if selector.startswith("//"):
#                                return driver.find_element(By.XPATH, selector).text
#                            else:
#                                return driver.find_element(By.CSS_SELECTOR, selector).text
#                        except:
#                            continue
#                    print(f"Could not find {element_type}")
#                    return "Not Found"
#                
#                # Try to get job details with multiple selectors
#                company_name = try_selectors(company_selectors, "company name")
#                
#                location_selectors = [
#                    ".css-56kyx5", 
#                    ".css-5vsc1i", 
#                    "[data-test='location']",
#                    "//div[contains(@class, 'location')]"
#                ]
#                location = try_selectors(location_selectors, "location")
#                
#                title_selectors = [
#                    ".css-1j389vi", 
#                    ".css-17x2pwl", 
#                    "[data-test='job-title']",
#                    "//div[contains(@class, 'title')]"
#                ]
#                job_title = try_selectors(title_selectors, "job title")
#                
#                desc_selectors = [
#                    ".jobDescriptionContent", 
#                    ".css-1lrhggr", 
#                    "[data-test='description']",
#                    "//div[contains(@class, 'descriptionContent')]"
#                ]
#                job_description = try_selectors(desc_selectors, "job description")
#                
#                salary_selectors = [
#                    ".css-1xe2xww", 
#                    ".css-16kxj2j", 
#                    "[data-test='salary-estimate']",
#                    "//span[contains(@class, 'salary')]"
#                ]
#                salary_estimate = try_selectors(salary_selectors, "salary")
#                
#                rating_selectors = [
#                    ".css-1m5m32b", 
#                    ".css-152xdkl", 
#                    "[data-test='rating']",
#                    "//span[contains(@class, 'rating')]"
#                ]
#                rating = try_selectors(rating_selectors, "rating")
#                
#            except Exception as e:
#                print(f"Error extracting job details: {e}")
#                continue
#
#            # Print for debugging
#            if verbose:
#                print(f"Job Title: {job_title}")
#                print(f"Company: {company_name}")
#                print(f"Location: {location}")
#                print(f"Salary: {salary_estimate}")
#                print(f"Rating: {rating}")
#                print("------------------------------------------------------")
#
#            # Only add the job if we found essential information
#            if job_title != "Not Found" and company_name != "Not Found":
#                jobs.append({
#                    "Job Title": job_title,
#                    "Company Name": company_name,
#                    "Location": location,
#                    "Salary Estimate": salary_estimate,
#                    "Rating": rating,
#                    "Job Description": job_description
#                })
#                print(f"Added job {len(jobs)}/{num_jobs}")
#
#        # If we've collected enough jobs, break out of the loop
#        if len(jobs) >= num_jobs:
#            break
#
#        # Try to navigate to the next page
#        try:
#            next_button_selectors = [
#                ".nextButton",
#                ".pagination__PaginationStyle__next",
#                "[data-test='pagination-next']",
#                "//button[contains(@class, 'next')]",
#                "//a[contains(@class, 'next')]",
#                "//button[contains(text(), 'Next')]",
#                "//button[contains(@aria-label, 'Next')]"
#            ]
#            
#            next_clicked = False
#            for selector in next_button_selectors:
#                try:
#                    if selector.startswith("//"):
#                        next_button = WebDriverWait(driver, 5).until(
#                            EC.element_to_be_clickable((By.XPATH, selector))
#                        )
#                    else:
#                        next_button = WebDriverWait(driver, 5).until(
#                            EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
#                        )
#                    
#                    driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
#                    time.sleep(1)
#                    next_button.click()
#                    next_clicked = True
#                    print(f"Clicked next button with selector: {selector}")
#                    break
#                except:
#                    continue
#            
#            if not next_clicked:
#                print("Could not find or click next button")
#                driver.save_screenshot("screenshots/no_next_button.png")
#                break
#                
#            page_num += 1
#            time.sleep(random.uniform(3.0, 5.0))  # Wait for next page to load
#            
#        except Exception as e:
#            print(f"Error navigating to next page: {e}")
#            driver.save_screenshot("screenshots/next_page_error.png")
#            break
#
#    print(f"Scraping finished. Found {len(jobs)} jobs.")
#    driver.quit()  # Close browser after scraping
#    
#    if len(jobs) == 0:
#        return pd.DataFrame()
#        
#    df = pd.DataFrame(jobs)
#    return df

In [ ]:
# Run the scraper for jobs in Canada
#df = get_jobs("Data Scientist", num_jobs=10, verbose=True)
#print(df.head())

Trying URL: https://www.glassdoor.ca/Job/canada-Data-Scientist-jobs-SRCH_IL.0,6_IN3_KO7,21.htm
Page title: 669 Data Scientist jobs in Canada, March 2025 | Glassdoor
Page source length: 995023
Looking for popup with selector: //button[contains(@class, 'close')]
Looking for popup with selector: //button[contains(@class, 'CloseButton')]
Looking for popup with selector: //span[contains(@class, 'CloseButton')]
Looking for popup with selector: //div[contains(@class, 'modal_closeIcon')]
Looking for popup with selector: //button[contains(text(), 'Close')]
Looking for popup with selector: //button[contains(text(), 'Continue')]
Looking for popup with selector: //button[contains(text(), 'Accept')]
Looking for job listings...
Trying selector: react-job-listing
Trying selector: JobCard
Trying selector: css-109wfo5
Trying selector: jobCard
Found 30 elements with selector: jobCard
Scraping page 1... Jobs collected so far: 0
Found 30 job cards on this page.
Clicking job card 1
Could not find job descr